<h3>Consine Similarity</h3>

<h4>Imports</h4>

In [1]:
import psycopg2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from psycopg2.extensions import register_adapter, AsIs
import numpy as np
import json

In [ ]:
conn = psycopg2.connect(
    host="my-db.c8684qazpg5m.us-east-1.rds.amazonaws.com",
    database="movie_recommendation_db",
    user="hemanth",
    password="hemanth123",
    port=5432
)
cur = conn.cursor()

# creating a tf-idf vectorizer for the movie titles and tags
tfidf = TfidfVectorizer(stop_words='english')
# getting movie titles and tags from the database
cur.execute("SELECT movieid, genres FROM movies")
movies = cur.fetchall()
# creating a list of movie titles and tags concatenated with space
movies_list = [movie[1] for movie in movies]
movies_list = [" ".join(s.split("|")) for s in movies_list]
# fit and transform the tfidf vectorizer on the movies list
tfidf_matrix = tfidf.fit_transform(movies_list)

In [ ]:
tfidf_matrix

In [ ]:
# computing cosine similarity matrix for the movies and store it in a table
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# create a table for cosine similarity matrix if not exists
# cur.execute("CREATE TABLE IF NOT EXISTS cosine_sim (movie_id INTEGER, sim_scores REAL[])")
conn.commit()

def adapt_numpy_array(numpy_array):
    return AsIs(tuple(numpy_array))

In [ ]:
# inserting records into table
register_adapter(np.ndarray, adapt_numpy_array)
for i in range(len(movies)):
    cur.execute("INSERT INTO cosine_sim (movie_id, sim_scores) VALUES (%s, %s)", (movies[i][0], json.dumps(cosine_sim[i].tolist())))
conn.commit()
cur.close()

In [ ]:
movies

In [2]:
conn = psycopg2.connect(
    host="my-db.c8684qazpg5m.us-east-1.rds.amazonaws.com",
    database="movie_recommendation_db",
    user="hemanth",
    password="hemanth123",
    port=5432
)

# creating a function to get the movie id from the movie title
def get_id_from_title(movie_title):
    with conn.cursor() as cur:
        # execute a SQL query 
        cur.execute("SELECT movieid FROM movies WHERE title ='"+movie_title+"'")
        # return the result
        return cur.fetchone()[0]
    

In [3]:
get_id_from_title('Toy Story (1995)')

1